In [ ]:
import sys
sys.path.append('../')
from SQLConnection import LocalConnection
from SQLConnection import AWSConnection
sys.path.append('../databases')
import mysql_aws_credentials as aws
from Format import Format
import re

In [ ]:
cons_test_users = aws_conn.get_users_by_pol_label('conservative')
libs_test_users = aws_conn.get_users_by_pol_label('liberal')
cons_train_users = local_conn.get_users_by_pol_label('conservative')
libs_train_users = local_conn.get_users_by_pol_label('liberal')

In [ ]:
print(cons_train_users.__len__())
print(libs_train_users.__len__())
cons_train_users[12]

In [ ]:
test_tweets = local_conn.get_all_tweets_by('sentedcruz')
test_tweets1 = local_conn.get_all_tweets_by('aoc')

In [ ]:
test_tweet = test_tweets[0][1]

In [ ]:
import nltk
nltk.download('stopwords')
import string

In [ ]:
denoised = []
denoised1 = []
for tweet in test_tweets:
    denoised.append(Format.denoise_tweet(tweet[1]))
    
for tweet in test_tweets1:
    denoised1.append(Format.denoise_tweet(tweet[1]))

In [ ]:
tokens = []
for i,d in enumerate(denoised):
    tokens.extend(d.split())
    
tokens1 = []
for i,d in enumerate(denoised1):
    print(i, test_tweets1[i][1])
    print('////')
    print(d,'\n')
    tokens1.extend(d.split())

In [ ]:
import collections
count = collections.Counter(tokens)
# count.most_common(50)

In [ ]:
import collections
count1 = collections.Counter(tokens1)
# count1.most_common(50)

In [ ]:
print(t, '\n')
t = Format.remove_hashtags(t)
print(t, '\n')
t = Format.remove_mentions(t)
print(t, '\n')
t = remove_stopwords(t)
print(t, '\n')
t = Format.remove_picture_links(t)
print(t, '\n')
t = t.translate(str.maketrans('','',string.punctuation + '—' + '“' + '…'))
print(t, '\n')
t.split()

In [ ]:
tweet_text = test_tweets[73][1]
tweet_text

In [ ]:
tweet_text = Format.remove_picture_links(tweet_text)
print(tweet_text, '\n')
tweet_text = tweet_text.lower()
print(tweet_text, '\n')
tweet_text = Format.remove_hashtags(tweet_text)
print(tweet_text, '\n')
tweet_text = Format.remove_mentions(tweet_text)
print(tweet_text, '\n')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import classification_report
from sklearn import svm 
from sklearn.model_selection import train_test_split

In [ ]:
cons_test_users = aws_conn.get_users_by_pol_label('conservative')
libs_test_users = aws_conn.get_users_by_pol_label('liberal')
cons_train_users = local_conn.get_users_by_pol_label('conservative')
libs_train_users = local_conn.get_users_by_pol_label('liberal')

In [ ]:
print(len(cons_test_users))
print(len(libs_test_users))
print(len(cons_train_users))
print(len(libs_train_users))

In [ ]:
# ('sentedcruz', 1074480192, '1074480192', 'conservative', None)
corpus = []
for i, user in enumerate(cons_train_users):
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0],tweets, user[3]))
    
for user in libs_train_users:
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))

In [ ]:
for i, user in enumerate(cons_test_users):
    tweets = aws_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))
    
for i, user in enumerate(libs_test_users):
    tweets = aws_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))

In [ ]:
len(corpus)


In [ ]:
data = []
labels = []
for row in corpus:
    data.append(row[1])
    labels.append((row[0], row[2]))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.1, random_state=6)

In [ ]:
y_train_username = []
y_train_label = []

for y in y_train:
    y_train_username.append(y[0])
    y_train_label.append(y[1])

In [ ]:
print(len(y_train_username))
print(len(y_train_label))

In [ ]:
print(len(y_test_username))
print(len(y_test_label))

In [ ]:
# data = test_data + train_data
# labels = test_labels + train_labels
# x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.33, random_state=42)

In [ ]:
# Create feature vectors 
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(x_train) #(train_data)
# Apply model on test data 
test_vectors = vectorizer.transform(x_test)#(test_data)

In [ ]:
# Perform classification with SVM, kernel=linear 
model = svm.SVC(kernel='linear') 
model.fit(train_vectors, y_train_label) 
prediction = model.predict(test_vectors)


In [ ]:
print(len(prediction))

In [ ]:
for i, pred in enumerate(prediction):
    if y_test_label[i] != pred:
        print(y_test_username[i], pred)

In [ ]:
print (classification_report(y_test_label, prediction))
# print(len(y_test_label))

# START


In [1]:
import sys
sys.path.append('../')
from SQLConnection import LocalConnection
from SQLConnection import AWSConnection
sys.path.append('../databases')
import mysql_aws_credentials as aws
from Format import Format
import re
import nltk
nltk.download('stopwords')
import string
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import classification_report
from sklearn import svm 
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/knownastron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/knownastron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# local_conn = LocalConnection('../databases/main.db')
aws_conn = AWSConnection(aws.HOST, aws.PORT, aws.DATABASE_NAME, aws.USER, aws.PASSWORD)

cons_test_users = aws_conn.get_users_by_pol_label('conservative')
libs_test_users = aws_conn.get_users_by_pol_label('liberal')
all_users = cons_test_users + libs_test_users


In [3]:
# corpus = []
# for i, user in enumerate(all_users):
#     print(i + 1, end=' ')
#     tweets = aws_conn.get_all_tweets_by(user[0])
#     tweets = [x[1] for x in tweets]
#     tweets = " ".join(tweets) #join all tweets into one big blob
#     tweets = Format.denoise_tweet(tweets)
#     corpus.append((user[0], tweets, user[3]))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [5]:
# joblib.dump(corpus, 'corpus.pkl')

['corpus.pkl']

In [10]:
corpus_object = open('corpus.pkl','rb')
corpus = joblib.load(corpus_object)


In [11]:
print(len(corpus))

630


In [12]:
data = []
labels = []
for row in corpus:
    data.append(row[1])
    labels.append((row[0], row[2])) #username and label

In [13]:
# corpus

In [14]:
print(len(data))

630


In [15]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.33, random_state=8)

In [16]:
#create lists of usernames and labels for training and test data
y_train_username = []
y_train_label = []

for y in y_train:
    y_train_username.append(y[0])
    y_train_label.append(y[1])
    
y_test_username = []
y_test_label = []

for y in y_test:
    y_test_username.append(y[0])
    y_test_label.append(y[1])

In [17]:
# Create feature vectors 
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(x_train) #(train_data)
# Apply model on test data 
test_vectors = vectorizer.transform(x_test) #(test_data)

In [18]:
test_vectors

<208x14569 sparse matrix of type '<class 'numpy.float64'>'
	with 332994 stored elements in Compressed Sparse Row format>

In [31]:
# Perform classification with SVM, kernel=rbf 
model_rbf = svm.SVC(kernel='rbf', gamma=1, C=50) 
model_rbf.fit(train_vectors, y_train_label) 
prediction_rbf = model_rbf.predict(test_vectors)

In [32]:
print(classification_report(y_test_label, prediction_rbf))

              precision    recall  f1-score   support

conservative       0.89      0.98      0.93       102
     liberal       0.98      0.89      0.93       106

    accuracy                           0.93       208
   macro avg       0.94      0.93      0.93       208
weighted avg       0.94      0.93      0.93       208



In [22]:
for i, pred in enumerate(prediction_rbf):
    if y_test_label[i] != pred:
        print(y_test_username[i], pred)

atrios conservative
eleanornorton conservative
repbrianfitz liberal
deray conservative
mudede conservative
senatorcollins liberal
yashar conservative
repaoc conservative
senatorsinema conservative
reprichmond conservative
staceyplaskett conservative
repschrader conservative
yamiche conservative
wonkroom conservative
senrickscott liberal
senatemajldr conservative
neonflag conservative
benniegthompson conservative
repdelgado conservative
moelleithee conservative
buzzfeedben conservative
sanfordbishop conservative
sendougjones conservative
repkendrahorn conservative
chrislhayes conservative
noahpinion conservative


In [25]:
# Perform classification with SVM, kernel=linear 
model_linear = svm.SVC(kernel='linear') 
model_linear.fit(train_vectors, y_train_label) 
prediction_linear = model_linear.predict(test_vectors)

In [26]:
print(classification_report(y_test_label, prediction_linear))

              precision    recall  f1-score   support

conservative       0.93      0.97      0.95       102
     liberal       0.97      0.92      0.95       106

    accuracy                           0.95       208
   macro avg       0.95      0.95      0.95       208
weighted avg       0.95      0.95      0.95       208



In [ ]:
for i, pred in enumerate(prediction):
    if y_test_label[i] != pred:
        print(y_test_username[i], pred)

# END

In [ ]:
# joblib.dump(model, 'NLP_political_classifier.pkl')
# joblib.dump(vectorizer, 'tfid_vectorizer.pkl')

In [ ]:
# nlp_model1 = open('NLP_political_classifier.pkl','rb')
# clf1 = joblib.load(nlp_model1)

# vectorizer_object1 = open('tfid_vectorizer.pkl', 'rb')
# vectorizer1 = joblib.load(vectorizer_object1)

vectorizer1 = vectorizer

In [ ]:
con_test_str = 'america is the land of the free home of the brave god bless'
lib_test_str = 'undocumented refugee climate change racism white people'
con_test_list = [Format.denoise_tweet(con_test_str)]
lib_test_list = [Format.denoise_tweet(lib_test_str)]
con_vecs = vectorizer1.transform(con_test_list)
lib_vecs = vectorizer1.transform(lib_test_list)

In [ ]:
prediction_con = model_rbf.predict(con_vecs)
prediction_lib = model_rbf.predict(lib_vecs)

In [ ]:
print(prediction_con)
print(prediction_lib)

In [ ]:
tweets = aws_conn.get_all_tweets_by('aoc')
tweets = [x[1] for x in tweets]
tweets = " ".join(tweets)
tweets = Format.denoise_tweet(tweets)

In [ ]:
tweets

In [ ]:
lib_test_list = [tweets]

In [ ]:
lib_test_vecs = vectorizer.transform(lib_test_list)
con_test_vecs = vectorizer.transform(con_test_list)

In [ ]:
ya = model.predict(lib_test_vecs)

In [ ]:
ya